# 00. Getting started with TensorFlow: A guide to the fundamentals

## What we're going to cover
TensorFlow is vast. But the main premise is simple: turn data into numbers (tensors) and build machine learning algorithms to find patterns in them.

In this notebook we cover some of the most fundamental TensorFlow operations, more specifically:

* Introduction to tensors (creating tensors)
* Getting information from tensors (tensor attributes)
* Manipulating tensors (tensor operations)
* Tensors and NumPy
* Using @tf.function (a way to speed up your regular Python functions)
* Using GPUs with TensorFlow
* Exercises to try

## Introduction to Tensors

In [1]:
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.8.2


## Creating Tensors with `tf.constant()`

In [2]:
scalar = tf.constant(2)
scalar.ndim, scalar

(0, <tf.Tensor: shape=(), dtype=int32, numpy=2>)

In [3]:
vector = tf.constant([1,2])
vector.ndim, vector

(1, <tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>)

In [4]:
matrix = tf.constant([[1,2,3],
                      [4,5,6]])
matrix.ndim, matrix

(2, <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>)

In [5]:
tensor = tf.constant([[[1, 2, 3],
                       [4, 5, 6]],
                      [[7, 8, 9],
                       [10, 11, 12]],
                      [[13, 14, 15],
                       [16, 17, 18]]])
tensor.ndim, tensor

(3, <tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
 array([[[ 1,  2,  3],
         [ 4,  5,  6]],
 
        [[ 7,  8,  9],
         [10, 11, 12]],
 
        [[13, 14, 15],
         [16, 17, 18]]], dtype=int32)>)

* **scalar**: a single number.
* **vector**: a number with direction (e.g. wind speed with direction).
* **matrix**: a 2-dimensional array of numbers.
* **tensor**: an n-dimensional array of numbers (where n∈N).

## Creating Tensors with `tf.Variable()`


In [6]:
changeable_tensor = tf.Variable([1,2])
unchangeable_tensor = tf.constant([1,2])

changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([1, 2], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>)

In [7]:
try :
    changeable_tensor[0] = 2
except TypeError :
    print("TypeError: 'ResourceVariable' object does not support item assignment")

TypeError: 'ResourceVariable' object does not support item assignment


In [8]:
changeable_tensor[0].assign(2) # inplace


<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([2, 2], dtype=int32)>

In [9]:
tensor = tf.Variable(np.arange(5))
tensor

<tf.Variable 'Variable:0' shape=(5,) dtype=int64, numpy=array([0, 1, 2, 3, 4])>

In [10]:
tensor.assign([0, 1, 2, 3, 40]) # inplace change

<tf.Variable 'UnreadVariable' shape=(5,) dtype=int64, numpy=array([ 0,  1,  2,  3, 40])>

In [11]:
tensor.assign_add([10, 10, 10, 10, 10]) # inplace change

<tf.Variable 'UnreadVariable' shape=(5,) dtype=int64, numpy=array([10, 11, 12, 13, 50])>

In [12]:
tensor

<tf.Variable 'Variable:0' shape=(5,) dtype=int64, numpy=array([10, 11, 12, 13, 50])>

## Creating random tensors

This is what neural networks use to **intialize their weights (patterns)** that they're trying to learn in the data.

We can create random tensors by using the 
<a href="https://www.tensorflow.org/guide/random_numbers#the_tfrandomgenerator_class">`tf.random.Generator`</a> 
class.

In [13]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(0) # set the seed for reproducibility
random_1 = random_1.normal(shape=(3, 2)) # create tensor from a normal distribution 
random_2 = tf.random.Generator.from_seed(0)
random_2 = random_2.normal(shape=(3, 2))

# Are they equal?
random_1, random_2, np.all(random_1 == random_2)

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3544159 ,  0.70454913],
        [ 0.03666191,  0.86918795],
        [ 0.43842277, -0.53439844]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3544159 ,  0.70454913],
        [ 0.03666191,  0.86918795],
        [ 0.43842277, -0.53439844]], dtype=float32)>,
 True)

In [14]:
g2 = tf.random.get_global_generator()
g2.normal(shape=[2, 3]) # different each time

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 0.5250821 ,  1.3795334 ,  1.6613544 ],
       [-0.51146865,  1.1496319 , -1.3104615 ]], dtype=float32)>

## Shuffle tensors

In [15]:
not_shuffle = tf.constant([[1,2],
                           [3,4],
                           [5,6],
                           [7,8]])

tf.random.shuffle(not_shuffle)

<tf.Tensor: shape=(4, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4],
       [5, 6],
       [7, 8]], dtype=int32)>

In [16]:
tf.random.shuffle(not_shuffle, seed=0) # operation level random seed

<tf.Tensor: shape=(4, 2), dtype=int32, numpy=
array([[3, 4],
       [5, 6],
       [1, 2],
       [7, 8]], dtype=int32)>

**You can also set the global seed using `tf.random.set_seed`**
<br>
**Note :** if both *global* and *operational* seed are set, both seeds are used in conjuction to determine the random sequence.

## Other way to create tensor

In [17]:
 tf.ones([4,8])

<tf.Tensor: shape=(4, 8), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [18]:
tf.zeros([4,8])

<tf.Tensor: shape=(4, 8), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

### Convert NumPy array to TenorFlow tensor

In [19]:
A_array = np.random.randint(0, 10, [4,8])

A_tensor = tf.constant(A_array)
A_tensor

<tf.Tensor: shape=(4, 8), dtype=int64, numpy=
array([[5, 2, 1, 8, 4, 6, 4, 3],
       [1, 6, 1, 0, 5, 7, 8, 0],
       [4, 2, 1, 1, 4, 9, 3, 1],
       [1, 9, 0, 9, 1, 6, 1, 4]])>

In [20]:
A_tensor_reshape = tf.constant(A_array, shape=(2,2,8))
A_tensor_reshape

<tf.Tensor: shape=(2, 2, 8), dtype=int64, numpy=
array([[[5, 2, 1, 8, 4, 6, 4, 3],
        [1, 6, 1, 0, 5, 7, 8, 0]],

       [[4, 2, 1, 1, 4, 9, 3, 1],
        [1, 9, 0, 9, 1, 6, 1, 4]]])>

**NOTE :** The purpose of tensor is that tensor can be run on GPU => way more faster 

### Convert TensorFlow tensor to NumPy array

In [21]:
A_tensor_reshape.numpy()

array([[[5, 2, 1, 8, 4, 6, 4, 3],
        [1, 6, 1, 0, 5, 7, 8, 0]],

       [[4, 2, 1, 1, 4, 9, 3, 1],
        [1, 9, 0, 9, 1, 6, 1, 4]]])

In [22]:
np.array(A_tensor_reshape)

array([[[5, 2, 1, 8, 4, 6, 4, 3],
        [1, 6, 1, 0, 5, 7, 8, 0]],

       [[4, 2, 1, 1, 4, 9, 3, 1],
        [1, 9, 0, 9, 1, 6, 1, 4]]])

## Getting information from tensors (shape, rank, size)

In [23]:
rank_4_tensor = tf.zeros([2, 3, 4, 5])

# Get various attributes of tensor
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank):", rank_4_tensor.ndim)
print("Shape of tensor:", rank_4_tensor.shape)
print("Elements along axis 0 of tensor:", rank_4_tensor.shape[0])
print("Elements along last axis of tensor:", rank_4_tensor.shape[-1])
print("Total number of elements (2*3*4*5):", tf.size(rank_4_tensor).numpy())

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tensor: (2, 3, 4, 5)
Elements along axis 0 of tensor: 2
Elements along last axis of tensor: 5
Total number of elements (2*3*4*5): 120


## Index Tensors AND reshape
You can also index tensors just like NumPy array.

In [24]:
random_tensor = tf.random.uniform(shape=(2,3,5), minval=1, maxval=5, dtype=tf.int32, seed=0)
random_tensor

<tf.Tensor: shape=(2, 3, 5), dtype=int32, numpy=
array([[[2, 2, 3, 2, 2],
        [3, 4, 3, 2, 2],
        [2, 3, 2, 3, 2]],

       [[3, 2, 4, 4, 3],
        [4, 2, 3, 1, 4],
        [2, 1, 4, 2, 2]]], dtype=int32)>

In [25]:
random_tensor[0]

<tf.Tensor: shape=(3, 5), dtype=int32, numpy=
array([[2, 2, 3, 2, 2],
       [3, 4, 3, 2, 2],
       [2, 3, 2, 3, 2]], dtype=int32)>

In [26]:
random_tensor[:,0]

<tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[2, 2, 3, 2, 2],
       [3, 2, 4, 4, 3]], dtype=int32)>

In [27]:
random_tensor[:,:,0]

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[2, 3, 2],
       [3, 4, 2]], dtype=int32)>

In [28]:
tf.reshape(random_tensor, [2,3,5,1])

<tf.Tensor: shape=(2, 3, 5, 1), dtype=int32, numpy=
array([[[[2],
         [2],
         [3],
         [2],
         [2]],

        [[3],
         [4],
         [3],
         [2],
         [2]],

        [[2],
         [3],
         [2],
         [3],
         [2]]],


       [[[3],
         [2],
         [4],
         [4],
         [3]],

        [[4],
         [2],
         [3],
         [1],
         [4]],

        [[2],
         [1],
         [4],
         [2],
         [2]]]], dtype=int32)>

## Manipulating tensors (tensor operations)

### Basic operations : `+` `-` `*` `/` 

In [29]:
tensor = tf.constant([[0,1],
                      [2,3]])

tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10, 11],
       [12, 13]], dtype=int32)>

In [30]:
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, 10],
       [20, 30]], dtype=int32)>

### In-build [functions](https://www.dummies.com/article/technology/information-technology/ai/machine-learning/create-basic-math-operations-tensorflow-253490/)

Using the TensorFlow function (where possible) has the advantage of being sped up later down the line when running as part of a 
[TensorFlow graph](https://www.tensorflow.org/tensorboard/graphs).

* Add
* Subtract
* Multiply
* Divide
* Square
* Reshape

In [31]:
tf.add(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10, 11],
       [12, 13]], dtype=int32)>

In [32]:
tf.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, 10],
       [20, 30]], dtype=int32)>

### [Math operation](https://www.tensorflow.org/api_docs/python/tf/math) : `tf.math`

In [33]:
# Ex :
tf.math.pow(tensor, 3)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0,  1],
       [ 8, 27]], dtype=int32)>

### Transpose : `tf.transpose`

In [34]:
tensor = tf.constant([[0, 1, 2],
                      [3, 4 ,5]])
tensor.shape

TensorShape([2, 3])

In [35]:
tensor = tf.transpose(tensor)
tensor, tensor.shape

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[0, 3],
        [1, 4],
        [2, 5]], dtype=int32)>, TensorShape([3, 2]))

### Basic function : `reduce_min` `reduce_max` `reduce_sum` `reduce_mean` etc...

In [36]:
tensor = tf.constant( np.random.randint(0, 100, (5, 10)), dtype=tf.float32)
tensor

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[ 1., 18., 10., 31.,  8., 15., 70., 30., 94., 21.],
       [76., 32., 23., 18., 17., 19., 83., 90., 88., 93.],
       [ 1., 35., 15., 54., 19., 17., 42., 22., 70., 90.],
       [61., 56., 17., 69., 39., 98., 53., 51.,  3., 29.],
       [79., 64., 72., 41., 95.,  3., 74., 18., 91., 22.]], dtype=float32)>

In [37]:
tf.reduce_max(tensor), tf.reduce_min(tensor)

(<tf.Tensor: shape=(), dtype=float32, numpy=98.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=1.0>)

In [38]:
tf.reduce_sum(tensor, axis=1), tf.reduce_sum(tensor)

(<tf.Tensor: shape=(5,), dtype=float32, numpy=array([298., 539., 365., 476., 559.], dtype=float32)>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2237.0>)

In [39]:
tf.reduce_mean(tensor)

<tf.Tensor: shape=(), dtype=float32, numpy=44.74>

In [40]:
tf.math.reduce_std(tensor, axis=1), tf.math.reduce_std(tensor) ## Warning, must be tf.float type

(<tf.Tensor: shape=(5,), dtype=float32, numpy=
 array([28.056372, 32.608124, 26.348625, 25.75733 , 30.907766],
       dtype=float32)>, <tf.Tensor: shape=(), dtype=float32, numpy=30.565214>)

In [41]:
tf.math.reduce_variance(tensor, axis=1), tf.math.reduce_variance(tensor)

(<tf.Tensor: shape=(5,), dtype=float32, numpy=
 array([ 787.16   , 1063.2899 ,  694.25   ,  663.44006,  955.29004],
       dtype=float32)>, <tf.Tensor: shape=(), dtype=float32, numpy=934.23236>)

In [42]:
import tensorflow_probability as tfp

tensor = tf.constant( np.random.randint(0, 100, (5, 10)))
tfp.stats.variance(tensor)

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([ 965,  279,  215, 1005,  625,  369,  492,  142,  922,  767])>

### Advance math function : Squaring, log, square root

* `tf.square()`
* `tf.sqrt()`
* `tf.math.log()`

In [43]:
tensor = tf.constant(np.random.randint(0, 10, (4,8)), dtype= tf.float32)
tensor

<tf.Tensor: shape=(4, 8), dtype=float32, numpy=
array([[7., 5., 7., 7., 2., 0., 2., 3.],
       [4., 2., 0., 4., 5., 7., 7., 0.],
       [9., 7., 8., 6., 3., 9., 5., 2.],
       [7., 0., 0., 8., 8., 3., 4., 6.]], dtype=float32)>

In [44]:
tf.square(tensor)

<tf.Tensor: shape=(4, 8), dtype=float32, numpy=
array([[49., 25., 49., 49.,  4.,  0.,  4.,  9.],
       [16.,  4.,  0., 16., 25., 49., 49.,  0.],
       [81., 49., 64., 36.,  9., 81., 25.,  4.],
       [49.,  0.,  0., 64., 64.,  9., 16., 36.]], dtype=float32)>

In [45]:
tf.sqrt(tensor)

<tf.Tensor: shape=(4, 8), dtype=float32, numpy=
array([[2.6457512, 2.2360678, 2.6457512, 2.6457512, 1.4142135, 0.       ,
        1.4142135, 1.7320508],
       [2.       , 1.4142135, 0.       , 2.       , 2.2360678, 2.6457512,
        2.6457512, 0.       ],
       [3.       , 2.6457512, 2.828427 , 2.4494896, 1.7320508, 3.       ,
        2.2360678, 1.4142135],
       [2.6457512, 0.       , 0.       , 2.828427 , 2.828427 , 1.7320508,
        2.       , 2.4494896]], dtype=float32)>

In [46]:
tf.math.log(tensor)

<tf.Tensor: shape=(4, 8), dtype=float32, numpy=
array([[1.9459102, 1.609438 , 1.9459102, 1.9459102, 0.6931472,      -inf,
        0.6931472, 1.0986123],
       [1.3862944, 0.6931472,      -inf, 1.3862944, 1.609438 , 1.9459102,
        1.9459102,      -inf],
       [2.1972246, 1.9459102, 2.0794415, 1.7917595, 1.0986123, 2.1972246,
        1.609438 , 0.6931472],
       [1.9459102,      -inf,      -inf, 2.0794415, 2.0794415, 1.0986123,
        1.3862944, 1.7917595]], dtype=float32)>

## Matrix mutliplication

You can either choose `tf.tensordot` or `tf.linalg.matmul` (short version : `tf.matmul` also work)<br>
**Note** : you can also use python function `@` : `tensor_1 @ tensor_2`

* [tensordot](https://www.tensorflow.org/api_docs/python/tf/tensordot)
* [matmul](https://www.tensorflow.org/api_docs/python/tf/linalg/matmul)
* [linalg](https://www.tensorflow.org/api_docs/python/tf/linalg)

<br>

**Rules for matrix multiplication :** <br>
let (a, b, c) be no-zeros interger :<br>
`(a, b) @ (b, c)` => `(a, c)`

In [47]:
tensor_1 = tf.constant([[0, 1, 2],
                        [3, 4, 5]])
tensor_2 = tf.constant([[0, 1],
                        [2, 3],
                        [5, 6]])

In [48]:
tf.tensordot(tensor_1, tensor_2, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[12, 15],
       [33, 45]], dtype=int32)>

In [49]:
tf.matmul(tensor_1, tensor_2)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[12, 15],
       [33, 45]], dtype=int32)>

In [50]:
tensor_1 @ tensor_2

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[12, 15],
       [33, 45]], dtype=int32)>

In [51]:
tf.matmul(a=tensor_2, b=tensor_1, transpose_a=True, transpose_b=True)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[12, 33],
       [15, 45]], dtype=int32)>

## Changing the datatype of a tensor

In [52]:
t1 = tf.constant([0,1])
t1.dtype 

tf.int32

In [53]:
t2 = tf.constant([0.,1.])
t2.dtype 

tf.float32

In [54]:
new_t1 = tf.cast(t1, tf.float16)
new_t1.dtype 

tf.float16

In [55]:
new_t2 = tf.cast(t2, tf.float32)
new_t2.dtype 

tf.float32

## Finding the positional maximum and minimum

* `tf.argmax()` - find the position of the maximum element in a given tensor.
* `tf.argmin()` - find the position of the minimum element in a given tensor.

In [56]:
tensor = tf.constant(np.random.randint(0, 100, (4,10)))
tensor

<tf.Tensor: shape=(4, 10), dtype=int64, numpy=
array([[92, 50, 92, 95, 57, 39,  8, 28, 25, 66],
       [77, 71,  8, 60, 90, 27, 41, 24, 59, 13],
       [21, 39, 26, 71,  9, 18, 22, 35, 52, 64],
       [54, 10, 60,  9, 30, 68, 41, 75, 47, 99]])>

In [57]:
tf.argmax(tensor, axis=1)

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([3, 4, 3, 9])>

In [58]:
tf.argmin(tensor, axis=0)

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([2, 3, 1, 3, 2, 2, 0, 1, 0, 1])>

## Squeezing a tensor (removing all single dimensions)
If you need to remove single-dimensions from a tensor (dimensions with size 1), you can use `tf.squeeze()`.

* `tf.squeeze()` - remove all dimensions of 1 from a tensor.

In [59]:
tensor = tf.constant(np.random.randint(0, 10, (5, 1)))
tensor

<tf.Tensor: shape=(5, 1), dtype=int64, numpy=
array([[0],
       [2],
       [4],
       [8],
       [5]])>

In [60]:
tf.squeeze(tensor)

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 2, 4, 8, 5])>

## One-hot encoding

If you have a tensor of indicies and would like to one-hot encode it, you can use 
[`tf.one_hot()`](https://www.tensorflow.org/api_docs/python/tf/one_hot)
.

You should also specify the depth parameter (the level which you want to one-hot encode to).

In [61]:
simple_list = [0, 1, 2, 3]
tf.one_hot(simple_list, 4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [62]:
simple_list = [0, 1, 2, 3, 2]
tf.one_hot(simple_list, 4)

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]], dtype=float32)>

## Using `@tf.function`

In your TensorFlow adventures, you might come across Python functions which have the decorator `@tf.function`.

But in short, decorators modify a function in one way or another.

In the `@tf.function` decorator case, it turns a Python function into a callable TensorFlow graph. Which is a fancy way of saying, if you've written your own Python function, and you decorate it with `@tf.function`, when you export your code (to potentially run on another device), TensorFlow will attempt to convert it into a fast(er) version of itself (by making it part of a computation graph).

For more on this, read the Better performnace with `tf.function` [guide](https://www.tensorflow.org/guide/function).

In [63]:
# Create a simple function
def function(x, y):
  return x ** 2 + y

In [64]:
# Create the same function and decorate it with tf.function
@tf.function
def tf_function(x, y):
  return x ** 2 + y

In [65]:
x = tf.constant(np.arange(0, 10))
y = tf.constant(np.arange(10, 20))

function(x, y)
tf_function(x, y)

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([ 10,  12,  16,  22,  30,  40,  52,  66,  82, 100])>

## Finding access to GPUs

In [66]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [67]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]